In [ ]:
#| include: false
#| default_exp glycan_data
import warnings
warnings.filterwarnings("ignore")
from nbdev.showdoc import show_doc
from IPython.display import HTML
import copy
#%load_ext autoreload
#%autoreload 2

In [ ]:
#| export
from glycowork.glycan_data.loader import *
from glycowork.glycan_data.loader import glycan_binding, df_glycan, df_species
from glycowork.glycan_data.stats import *
from glycowork.glycan_data.data_entry import *

`glycan_data` contains several helper functions for glycan data loading and data entry. Helper functions for data loading and data objects are in `loader` and include:

- `unwrap` flattens nested lists
- `find_nth` returns the n-th occurrence of a motif
- `df_species` loaded file for all glycans with species information, one row per glycan-species combination
- `df_glycan` loaded file containing all unique glycans with lots of meta-information
- `glycan_motifs` literature-known glycan motifs together with their sequence
- `glycan_binding` Z-score data for >2,000 lectins binding to glycans; every row one protein, glycans are columns
- `lib` library of unique monosaccharides + linkages in stored datasets; generated with `get_lib` & stored pre-computed

Further, the `stats` module contains many helper functions for statistics and data normalization.

## loader

In [ ]:
show_doc(unwrap)

---

### unwrap

>      unwrap (nested_list:list[typing.Any])

*converts a nested list into a flat list*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| nested_list | list | list to be flattened |
| **Returns** | **list** | **flattened list** |

In [ ]:
unwrap([[1,2], [3,4]])

[1, 2, 3, 4]

In [ ]:
show_doc(find_nth)

---

### find_nth

>      find_nth (haystack:str, needle:str, n:int)

*finds n-th instance of motif*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| haystack | str | string to search for motif |
| needle | str | motif |
| n | int | n-th occurrence in string (not zero-indexed) |
| **Returns** | **int** | **starting index of n-th occurrence** |

In [ ]:
find_nth('This is as good as it gets', 'as', 2)

16

In [ ]:
show_doc(find_nth_reverse)

---

### find_nth_reverse

>      find_nth_reverse (string:str, substring:str, n:int,
>                        ignore_branches:bool=False)

*finds n-th instance of motif from end of string*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| string | str |  | string to search |
| substring | str |  | substring to find |
| n | int |  | n-th occurrence from end |
| ignore_branches | bool | False | whether to ignore branches when counting |
| **Returns** | **int** |  | **position of n-th occurrence from end** |

In [ ]:
find_nth_reverse("Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc", "Gal", 0)

38

In [ ]:
show_doc(replace_every_second)

---

### replace_every_second

>      replace_every_second (string:str, old_char:str, new_char:str)

*function to replace every second occurrence of old_char in string with new_char*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| string | str | input string |
| old_char | str | character to replace |
| new_char | str | character to replace with |
| **Returns** | **str** | **modified string** |

In [ ]:
replace_every_second("Neu5Ac(a2-3(Gal(b1-4(Glc", '(', ')')

'Neu5Ac(a2-3)Gal(b1-4)Glc'

In [ ]:
show_doc(multireplace)

---

### multireplace

>      multireplace (string:str, remove_dic:dict[str,str])

*Replaces all occurences of items in a string with a given string*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| string | str | string to perform replacements on |
| remove_dic | dict | dict of form to_replace:replace_with |
| **Returns** | **str** | **modified string** |

In [ ]:
multireplace("Neu5Ac[a2-3]Gal", {'[': '(', ']': ')'})

'Neu5Ac(a2-3)Gal'

In [ ]:
show_doc(build_custom_df)

---

### build_custom_df

>      build_custom_df (df:pandas.core.frame.DataFrame, kind:str='df_species')

*creates custom df from df_glycan*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DataFrame |  | df_glycan / sugarbase |
| kind | str | df_species | whether to create 'df_species', 'df_tissue', or 'df_disease' |
| **Returns** | **DataFrame** |  | **custom df with one glycan - species/tissue/disease association per row** |

In [ ]:
build_custom_df(df_glycan, kind = 'df_tissue')

,glycan,tissue_sample,tissue_species,tissue_id,tissue_ref
0,Fuc(a1-2)Gal(b1-4)GlcNAc(b1-2)Man(a1-3)[Man(a1...,2A3_cell_line,Homo_sapiens,cellosaurus:CVCL_0D71,https://pubmed.ncbi.nlm.nih.gov/36289103/
1,Fuc(a1-3)[Gal(b1-4)]GlcNAc(b1-2)Man(a1-3)[Man(...,2A3_cell_line,Homo_sapiens,cellosaurus:CVCL_0D71,https://pubmed.ncbi.nlm.nih.gov/36289103/
2,Gal(b1-4)GlcNAc(b1-2)Man(a1-3)[Gal(b1-4)GlcNAc...,2A3_cell_line,Homo_sapiens,cellosaurus:CVCL_0D71,https://pubmed.ncbi.nlm.nih.gov/36289103/
3,Gal(b1-4)GlcNAc(b1-2)Man(a1-3)[Gal(b1-4)GlcNAc...,2A3_cell_line,Homo_sapiens,cellosaurus:CVCL_0D71,https://pubmed.ncbi.nlm.nih.gov/36289103/
4,Gal(b1-4)GlcNAc(b1-2)Man(a1-3)[Man(a1-3)[Man(a...,2A3_cell_line,Homo_sapiens,cellosaurus:CVCL_0D71,https://pubmed.ncbi.nlm.nih.gov/36289103/
...,...,...,...,...,...
20076,Neu5Ac(a2-8)Neu5Ac(a2-3)Gal(b1-4)Glc1Cer,zone_of_skin,Xiphophorus_sp,UBERON:0000014,https://pubmed.ncbi.nlm.nih.gov/3370642/
20077,Neu5AcOAc(a2-8)Neu5Ac(a2-3)Gal(b1-4)Glc1Cer,zone_of_skin,Xiphophorus_sp,UBERON:0000014,https://pubmed.ncbi.nlm.nih.gov/3370642/
20078,{Fuc(a1-2/3/4)}Neu5Ac(a2-3/6)Gal(b1-4)GlcNAc(b...,zone_of_skin,Homo_sapiens,UBERON:0000014,http://www.functionalglycomics.org/glycomics/S...
20079,{Man(a1-2)}Man(a1-2)Man(a1-3)[Man(a1-3)[Man(a1...,zone_of_skin,Homo_sapiens,UBERON:0000014,http://www.functionalglycomics.org/glycomics/S...


`df_species`

In [ ]:
#| echo: false
df_species2 = copy.deepcopy(df_species)
df_species2.set_index("glycan", inplace = True)
df_species2.head().style.set_properties(**{'font-size': '11pt', 'font-family': 'Helvetica','border-collapse': 'collapse','border': '1px solid black'})

,Species,Genus,Family,Order,Class,Phylum,Kingdom,Domain,ref
glycan,,,,,,,,,
Gal(a1-2)[Rha3Me(a1-3)][Xyl(b1-4)]Fuc(a1-3)[Xyl(b1-4)]Glc,ATCV-1,Chlorovirus,Phycodnaviridae,Algavirales,Megaviricetes,Nucleocytoviricota,Bamfordvirae,Virus,https://pubmed.ncbi.nlm.nih.gov/26582281/
Gal(a1-2)[Rha3Me(a1-3)][Xyl(b1-4)]Fuc(a1-3)[Xyl4Me(b1-4)]Glc,ATCV-1,Chlorovirus,Phycodnaviridae,Algavirales,Megaviricetes,Nucleocytoviricota,Bamfordvirae,Virus,https://pubmed.ncbi.nlm.nih.gov/26582281/
Gal(a1-2)[Rha3Me(a1-3)][Xyl4Me(b1-4)]Fuc(a1-3)[Xyl(b1-4)]Glc,ATCV-1,Chlorovirus,Phycodnaviridae,Algavirales,Megaviricetes,Nucleocytoviricota,Bamfordvirae,Virus,https://pubmed.ncbi.nlm.nih.gov/26582281/
Gal(a1-2)[Rha3Me(a1-3)][Xyl4Me(b1-4)]Fuc(a1-3)[Xyl4Me(b1-4)]Glc,ATCV-1,Chlorovirus,Phycodnaviridae,Algavirales,Megaviricetes,Nucleocytoviricota,Bamfordvirae,Virus,https://pubmed.ncbi.nlm.nih.gov/26582281/
GalA(a1-2)Rha(a1-4)GalA,Abelmoschus_esculentus,Abelmoschus,Malvaceae,Malvales,Dicotyledons,Angiosperms,Plantae,Eukarya,nan


In [ ]:
df_species.glyco_filter("Neu5Ac(a2-3)[GalNAc(b1-4)]Gal", min_count = 2)

,glycan,Species,Genus,Family,Order,Class,Phylum,Kingdom,Domain,ref
0,Neu5Ac(a2-3)[GalNAc(b1-4)]Gal(b1-3)GalNAc(b1-4...,Bos_taurus,Bos,Bovidae,Artiodactyla,Mammalia,Chordata,Animalia,Eukarya,gwb
1,Neu5Ac(a2-3)[GalNAc(b1-4)]Gal(b1-4)GlcNAc(b1-6...,Diceros_bicornis,Diceros,Rhinocerotidae,Perissodactyla,Mammalia,Chordata,Animalia,Eukarya,https://www.biorxiv.org/content/10.1101/2023.0...
2,Neu5Ac(a2-3)[GalNAc(b1-4)]Gal(b1-4)GlcNAc(b1-6...,Diceros_bicornis,Diceros,Rhinocerotidae,Perissodactyla,Mammalia,Chordata,Animalia,Eukarya,https://www.biorxiv.org/content/10.1101/2023.0...
3,Neu5Ac(a2-3)[GalNAc(b1-4)]Gal(b1-3)GalNAc(b1-4...,Homo_sapiens,Homo,Hominidae,Primates,Mammalia,Chordata,Animalia,Eukarya,NaN
4,Neu5Ac(a2-3)[GalNAc(b1-4)]Gal(b1-3/4)GlcNAc(b1...,Homo_sapiens,Homo,Hominidae,Primates,Mammalia,Chordata,Animalia,Eukarya,https://pubmed.ncbi.nlm.nih.gov/32236654/
5,Neu5Ac(a2-3)[GalNAc(b1-4)]Gal(b1-4)GlcNAc(b1-2...,Homo_sapiens,Homo,Hominidae,Primates,Mammalia,Chordata,Animalia,Eukarya,https://pubmed.ncbi.nlm.nih.gov/33650863/
6,Neu5Ac(a2-3)[GalNAc(b1-4)]Gal(b1-4)GlcNAc(b1-2...,Homo_sapiens,Homo,Hominidae,Primates,Mammalia,Chordata,Animalia,Eukarya,https://pubmed.ncbi.nlm.nih.gov/33650863/
7,Neu5Ac(a2-3)[GalNAc(b1-4)]Gal(b1-4)GlcNAc(b1-2...,Homo_sapiens,Homo,Hominidae,Primates,Mammalia,Chordata,Animalia,Eukarya,https://pubmed.ncbi.nlm.nih.gov/33650863/
8,Neu5Ac(a2-3)[GalNAc(b1-4)]Gal(b1-4)GlcNAc(b1-2...,Homo_sapiens,Homo,Hominidae,Primates,Mammalia,Chordata,Animalia,Eukarya,https://pubmed.ncbi.nlm.nih.gov/33650863/
9,Neu5Ac(a2-3)[GalNAc(b1-4)]Gal(b1-4)GlcNAc(b1-2...,Homo_sapiens,Homo,Hominidae,Primates,Mammalia,Chordata,Animalia,Eukarya,https://pubmed.ncbi.nlm.nih.gov/33650863/


`df_glycan`

In [ ]:
#| echo: false
df_glycan2 = copy.deepcopy(df_glycan)
df_glycan2.set_index("glycan", inplace = True)
df_glycan2.head().style.set_properties(**{'font-size': '8pt', 'font-family': 'Helvetica','border-collapse': 'collapse','border': '1px solid black'})

`glycan_binding`

In [ ]:
#| echo: false
glycan_binding.head().style.set_properties(**{'font-size': '11pt', 'font-family': 'Helvetica','border-collapse': 'collapse','border': '1px solid black'})

## stats

In [ ]:
show_doc(get_BF)

---

### get_BF

>      get_BF (n:int, p:float, z:bool=False, method:str='robust',
>              upper:float=10)

*Transforms a p-value into Jeffreys' approximate Bayes factor (BF)*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| n | int |  | sample size |
| p | float |  | p-value |
| z | bool | False | True if p-value from z-statistic, False if t-statistic |
| method | str | robust | method for choice of 'b': "JAB", "min", "robust", "balanced" |
| upper | float | 10 | upper limit for range of realistic effect sizes |
| **Returns** | **float** |  | **Bayes factor in favor of H1** |

In [ ]:
get_BF(6, 0.05)

30.1559980528856

In [ ]:
show_doc(get_alphaN)

---

### get_alphaN

>      get_alphaN (n:int, BF:float=3, method:str='robust', upper:float=10)

*Set the alpha level based on sample size via Bayesian-Adaptive Alpha Adjustment*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| n | int |  | sample size |
| BF | float | 3 | Bayes factor you would like to match |
| method | str | robust | method for choice of 'b': "JAB", "min", "robust", "balanced" |
| upper | float | 10 | upper limit for range of realistic effect sizes |
| **Returns** | **float** |  | **alpha level required to achieve desired evidence** |

In [ ]:
get_alphaN(6)

You're working with an alpha of 0.07862467893233027 that has been adjusted for your sample size of 6.


0.07862467893233027

In [ ]:
show_doc(TST_grouped_benjamini_hochberg)

---

### TST_grouped_benjamini_hochberg

>      TST_grouped_benjamini_hochberg (identifiers_grouped:dict[str,list],
>                                      p_values_grouped:dict[str,list[float]],
>                                      alpha:float)

*perform the two-stage adaptive Benjamini-Hochberg procedure for multiple testing correction*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| identifiers_grouped | dict | dictionary of group : list of glycans |
| p_values_grouped | dict | dictionary of group : list of p-values |
| alpha | float | significance threshold for testing |
| **Returns** | **tuple** | **(glycan:corrected p-value dict, glycan:significant dict)** |

In [ ]:
grouped_glycans = {'core2': ['Fuc(a1-2)Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc', 'Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc', 'Neu5Ac(a2-3)Gal(b1-3)[Gal(b1-4)GlcNAc(b1-6)]GalNAc', 'Neu5Ac(a2-3)Gal(b1-3)[Gal(b1-4)GlcNAc6S(b1-6)]GalNAc', 'Neu5Ac(a2-3)Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc', 'Neu5Ac(a2-3)Gal(b1-4)GlcNAc(b1-6)[Neu5Ac(a2-3)Gal(b1-3)]GalNAc', 'Neu5Ac(a2-3)Gal(b1-4)GlcNAc6S(b1-6)[Gal(b1-3)]GalNAc', 'Neu5Ac(a2-3)Gal(b1-4)GlcNAc6S(b1-6)[Neu5Ac(a2-3)Gal(b1-3)]GalNAc'], 'core1': ['Gal(b1-3)[Neu5Ac(a2-6)]GalNAc', 'GalOS(b1-3)[Neu5Ac(a2-6)]GalNAc', 'Neu5Ac(a2-3)Gal(b1-3)GalNAc', 'Neu5Ac(a2-3)Gal(b1-3)[Neu5Ac(a2-6)]GalNAc', 'Neu5Ac(a2-3)Gal(b1-3)[Neu5Ac(a2-8)Neu5Ac(a2-6)]GalNAc']}
grouped_pvals = {'core2': [0.6142626731590599, 0.7070153617674607, 0.019898944554207577, 0.8026651973502792, 0.015092570600834338, 0.0023818885119090837, 0.5611180077391343, 0.005416655300198711], 'core1': [0.38054124309577675, 0.0003019282861611496, 0.03584424395333232, 0.010311562841413265, 0.7501582090523815]}
alpha = 0.054572814701647765
TST_grouped_benjamini_hochberg(grouped_glycans, grouped_pvals, alpha)

({'Fuc(a1-2)Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc': 0.8026651973502792,
  'Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc': 0.8026651973502792,
  'Neu5Ac(a2-3)Gal(b1-3)[Gal(b1-4)GlcNAc(b1-6)]GalNAc': 0.039797889108415155,
  'Neu5Ac(a2-3)Gal(b1-3)[Gal(b1-4)GlcNAc6S(b1-6)]GalNAc': 0.8026651973502792,
  'Neu5Ac(a2-3)Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc': 0.039797889108415155,
  'Neu5Ac(a2-3)Gal(b1-4)GlcNAc(b1-6)[Neu5Ac(a2-3)Gal(b1-3)]GalNAc': 0.01905510809527267,
  'Neu5Ac(a2-3)Gal(b1-4)GlcNAc6S(b1-6)[Gal(b1-3)]GalNAc': 0.8026651973502792,
  'Neu5Ac(a2-3)Gal(b1-4)GlcNAc6S(b1-6)[Neu5Ac(a2-3)Gal(b1-3)]GalNAc': 0.021666621200794843,
  'Gal(b1-3)[Neu5Ac(a2-6)]GalNAc': 0.4756765538697209,
  'GalOS(b1-3)[Neu5Ac(a2-6)]GalNAc': 0.001509641430805748,
  'Neu5Ac(a2-3)Gal(b1-3)GalNAc': 0.059740406588887206,
  'Neu5Ac(a2-3)Gal(b1-3)[Neu5Ac(a2-6)]GalNAc': 0.02577890710353316,
  'Neu5Ac(a2-3)Gal(b1-3)[Neu5Ac(a2-8)Neu5Ac(a2-6)]GalNAc': 0.7501582090523815},
 {'Fuc(a1-2)Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc': 

## data_entry

Helper functions for entering data are contained in `data_entry` and include:

- `check_presence` tests whether the entered glycan is already in database

In [ ]:
show_doc(check_presence)

---

### check_presence

>      check_presence (glycan:str, df:pandas.core.frame.DataFrame,
>                      colname:str='glycan', name:str|None=None,
>                      rank:str='Species', fast:bool=False)

*checks whether glycan (of that species) is already present in dataset*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| glycan | str |  | IUPAC-condensed glycan sequence |
| df | DataFrame |  | glycan dataframe where glycans are under colname |
| colname | str | glycan | column name containing glycans |
| name | str \| None | None | name of species of interest |
| rank | str | Species | column name for filtering |
| fast | bool | False | True uses precomputed glycan graphs |
| **Returns** | **None** |  |  |

In [ ]:
print("Check presence of Man(a1-3)[Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc")
check_presence('Man(a1-3)[Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc', df_species)

print("Check presence of Fuc(b1-3)[Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc")
check_presence('Fuc(b1-3)[Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc', df_species)

print("Check presence of Man(a1-3)[Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc in the species Danielus Bojarum")
check_presence('Man(a1-3)[Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc', df_species,
               name = 'Danielus Bojarum')

Check presence of Man(a1-3)[Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc
Glycan already in dataset.
Check presence of Fuc(b1-3)[Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc
It's your lucky day, this glycan is new!
Check presence of Man(a1-3)[Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc in the species Danielus Bojarum
This is the best: Danielus_Bojarum is not in dataset
It's your lucky day, this glycan is new!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()